In [1]:
from langchain.llms import GooglePalm
api_key = "....Enter your Google Palm API key...."

llm = GooglePalm(google_api_key= api_key, temperature= 0.3)

poem = llm("Give me a joke")
print(poem)

Why did the scarecrow win an award?

Because he was outstanding in his field!


In [2]:
from langchain.utilities import SQLDatabase

from urllib.parse import quote



db_user = "root"
db_password = "Cooldude@1"
encoded_password = quote(db_password)
db_host = "127.0.0.1"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{encoded_password}@{db_host}/{db_name}" )
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [3]:
from langchain.base_language import BaseLanguageModel
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose = True)
qns1 = db_chain.run("How many tshirts are there in nike company which are white in color and medium size")



> Entering new SQLDatabaseChain chain...
How many tshirts are there in nike company which are white in color and medium size
SQLQuery:SELECT count(*) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'M'
SQLResult: [(1,)]
Answer:1
> Finished chain.


In [4]:
qns1

'1'

In [5]:
qns2 = db_chain.run("What is the total cost of all small size tshirts")



> Entering new SQLDatabaseChain chain...
What is the total cost of all small size tshirts
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('336'),)]
Answer:336
> Finished chain.


In [6]:
qns2 = db_chain.run("select sum(price*stock_quantity) as total_price from t_shirts where size = 'S';-- ignore the comments")




> Entering new SQLDatabaseChain chain...
select sum(price*stock_quantity) as total_price from t_shirts where size = 'S';-- ignore the comments
SQLQuery:SELECT sum(price*stock_quantity) AS total_price FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('16894'),)]
Answer:16894
> Finished chain.


In [8]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (100 - pct_discount) / 100)
FROM discounts
JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id
WHERE brand = 'Levi' AND t_shirts.stock_quantity > 0
SQLResult: [(Decimal('105.100000'),)]
Answer:105.1
> Finished chain.


In [12]:
qns3 = db_chain.run("""select sum((t.t_shirts_price * (100.00 - d.pct_discount)/100)) as total_rev_post_discount from discounts d
left join (select (price*stock_quantity) as t_shirts_price, t_shirt_id from t_shirts where brand = 'Levi') t on d.t_shirt_id = t.t_shirt_id
""")




> Entering new SQLDatabaseChain chain...
select sum((t.t_shirts_price * (100.00 - d.pct_discount)/100)) as total_rev_post_discount from discounts d
left join (select (price*stock_quantity) as t_shirts_price, t_shirt_id from t_shirts where brand = 'Levi') t on d.t_shirt_id = t.t_shirt_id

SQLQuery:select sum((t.t_shirts_price * (100.00 - d.pct_discount)/100)) as total_rev_post_discount from discounts d
left join (select (price*stock_quantity) as t_shirts_price, t_shirt_id from t_shirts where brand = 'Levi') t on d.t_shirt_id = t.t_shirt_id
SQLResult: [(Decimal('5571.500000'),)]
Answer:5571.50
> Finished chain.


In [13]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")




> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('19249'),)]
Answer:19249
> Finished chain.


In [15]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")




> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('142'),)]
Answer:142
> Finished chain.


### Few shot learning

In [16]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in M size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'M'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """select sum((t.t_shirts_price * (100.00 - d.pct_discount)/100)) as total_rev_post_discount from discounts d 
     left join (select (price*stock_quantity) as t_shirts_price, t_shirt_id from t_shirts where brand = 'Levi') t on d.t_shirt_id = t.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [33]:
few_shots

[{'Question': 'How many t-shirts do we have left for Nike in M size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'M'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '1'},
 {'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of the SQL query',
  'Answer': '16894'},
 {'Question': 'If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
  'SQLQuery': "select sum((t.t_shirts_price * (100.00 - d.pct_discount)/100)) as total_rev_post_discount from discounts d \n     left join (select (price*stock_quantity) as t_shirts_price, t_shirt_id from t_shirts where brand = 'Levi') t on d.t_shirt_id = t.t_shirt_id\n ",
  'SQLResult': 'Result of the SQL query',
  'Answer': '5571.50'},
 {'Question': 'If

In [17]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

In [18]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [19]:
vectorstore

In [20]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '1',
  'Question': 'How many t-shirts do we have left for Nike in M size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'M'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '19249',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [21]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [22]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [23]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [24]:

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [25]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [26]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)


In [27]:
new_chain("How many white color Levi's shirt I have?")




> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('142'),)]
Answer:142
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '142'}

In [28]:
new_chain("How much is the price of the inventory for all small size t-shirts?")




> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('16894'),)]
Answer:16894
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '16894'}

In [29]:
new_chain("How much is the price of all white color levi t shirts?")




> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('2080'),)]
Answer:2080
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': '2080'}

In [30]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")




> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:select sum((t.t_shirts_price * (100.00 - d.pct_discount)/100)) as total_rev_post_discount from discounts d 
    left join (select (price*stock_quantity) as t_shirts_price, t_shirt_id from t_shirts where brand = 'Nike') t on d.t_shirt_id = t.t_shirt_id
SQLResult: [(Decimal('1787.500000'),)]
Answer:1787.5
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '1787.5'}

In [31]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")




> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:select sum((t.t_shirts_price * (100.00 - d.pct_discount)/100)) as total_rev_post_discount from discounts d 
    left join (select (price*stock_quantity) as t_shirts_price, t_shirt_id from t_shirts where brand = 'Van Huesen') t on d.t_shirt_id = t.t_shirt_id
SQLResult: [(Decimal('2641.400000'),)]
Answer:2641.4
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '2641.4'}

In [32]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('18684'),)]
Answer:18684
> Finished chain.


'18684'